In [ ]:
# Load image 
# Split image into range and domain blocks
# Apply transformations to each block
# Compare range to domain blocks using metric
# Store transformations for each domain block
# Use Partitioned IFS to recreate image

In [116]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import DomainBlock as db
import cv2

In [117]:
class Partition:

    def __init__(self, image, block_size):
        
        self.rows, self.cols = image.shape

        if self.rows != self.cols:
            raise NotImplementedError("Only encodes square images")

        if self.rows % block_size != 0 or self.cols % block_size != 0:
            raise NotImplementedError("Image dimensions must be divisible by block size")
        
        self.block_size = block_size
        self.blocks_per_row = self.rows // self.block_size
        self.blocks_per_col = self.cols // self.block_size

        self.num_blocks = self.blocks_per_row * self.blocks_per_col
    
    def generate_blocks(self):
        domain_blocks = []

        for b in range(self.num_blocks):
            i_top_left = (b // self.blocks_per_row) * self.block_size
            j_top_left = (b % self.blocks_per_col) * self.block_size
            
            i_bot_right = i_top_left + self.block_size
            j_bot_right = j_top_left + self.block_size

            domain_blocks.append(db.DomainBlock((i_top_left, j_top_left), (i_bot_right, j_bot_right)))

        return domain_blocks


In [139]:
class ImageCompression():

    def __init__(self, image, domainBlocks, rangeBlocks):
        self.image = image
        self.domainBlocks = domainBlocks
        self.rangeBlocks = rangeBlocks
        self.ifs_list = []
        possTransforms = self.generate_all_transforms()
        self.domainBlockTransforms = self.transform_all_blocks(possTransforms)

    def generate_all_transforms(self):
    
        transforms = []

        for rotation in [0, 90, 180, 270]:
            for flip in [True, False]:
                transforms.append((flip, rotation))

        return transforms
    
    def transform_all_blocks(self, possTransforms):

        transforms = []

        for b in self.domainBlocks:

            transforms.extend(self.block_transforms(b, possTransforms))

        return transforms


    def block_transforms(self, domainBlock, possTransforms):

        transform_list = []

        top_left = domainBlock.top_left
        bot_right = domainBlock.bot_right

        domain_img = self.image[top_left[0]:bot_right[0], top_left[1]:bot_right[1]].copy()

        for flip, rotation in possTransforms:
            transformed = domain_img.copy()

            if flip:
                transformed = np.fliplr(transformed)
            
            currentRotation = 0

            while currentRotation < rotation:
                transformed = np.rot90(transformed)
                currentRotation += 90
            
            transform_list.append((domainBlock, flip, rotation, transformed))
        
        return transform_list
    
    def compute_diff(self, range, domain, s, o):
        # currently using frobenius norm
        return np.sum(np.square(range - ((s * domain) + o)))
    
    def compute_s_o(self, range, domain):

        rows, cols = domain.shape

        n = rows * cols

        s = self.best_s(range, domain, n)
        o = self.best_o(range, domain, n, s)

        return s, o

        
    def best_s(self, range, domain, n):

        # Optimal alpha shown in Yuval Fisher book

        top_frac = (n * np.sum(range * domain)) - (np.sum(range) * np.sum(domain))

        bot_frac = (n * np.sum(np.square(domain))) - (np.sum(domain))^2

        return top_frac / bot_frac

    def best_o(self, range, domain, n, s):

        return (1/ n) * (np.sum(range) - s * np.sum(domain))
        
    def find_best_transform(self, rangeBlock):

        current_min = np.inf

        range_image = self.image[rangeBlock.top_left[0]:rangeBlock.bot_right[0], rangeBlock.top_left[1]:rangeBlock.bot_right[1]]

        for domainBlock, flip, rotation, transformation in self.domainBlockTransforms:


            s, o = self.compute_s_o(range_image, transformation)

            difference = self.compute_diff(range_image, transformation, s, o)

            if difference < current_min:

                current_min = difference
                best_contraction = (domainBlock, flip, rotation, s, o)

        return best_contraction

    def encode(self):

        encoded = []

        for i, rangeBlock in enumerate(self.rangeBlocks):

            print(f"Encoding block {i + 1} of {len(self.rangeBlocks)}")
            
            code_block = self.find_best_transform(rangeBlock)

            encoded.append(code_block)
        
        return encoded
    

In [138]:
image = cv2.imread("lena.jpg", 0)

range_image = cv2.resize(image, (image.shape[1] // 4, image.shape[0] // 4))
domain_image = cv2.resize(image, (image.shape[1] // 8, image.shape[0] // 8))

range_partition = Partition(range_image, 8).generate_blocks()
domain_partition = Partition(domain_image, 8).generate_blocks()
                         
compression = ImageCompression(range_image, domain_partition, range_partition)

compression.encode()


Block 0 of 256
Block 1 of 256
Block 2 of 256
Block 3 of 256
Block 4 of 256
Block 5 of 256
Block 6 of 256
Block 7 of 256
Block 8 of 256
Block 9 of 256
Block 10 of 256
Block 11 of 256
Block 12 of 256
Block 13 of 256
Block 14 of 256
Block 15 of 256
Block 16 of 256
Block 17 of 256
Block 18 of 256
Block 19 of 256
Block 20 of 256
Block 21 of 256
Block 22 of 256
Block 23 of 256
Block 24 of 256
Block 25 of 256
Block 26 of 256
Block 27 of 256
Block 28 of 256
Block 29 of 256
Block 30 of 256
Block 31 of 256
Block 32 of 256
Block 33 of 256
Block 34 of 256
Block 35 of 256
Block 36 of 256
Block 37 of 256
Block 38 of 256
Block 39 of 256
Block 40 of 256
Block 41 of 256
Block 42 of 256
Block 43 of 256
Block 44 of 256
Block 45 of 256
Block 46 of 256
Block 47 of 256
Block 48 of 256
Block 49 of 256
Block 50 of 256
Block 51 of 256
Block 52 of 256
Block 53 of 256
Block 54 of 256
Block 55 of 256
Block 56 of 256
Block 57 of 256
Block 58 of 256
Block 59 of 256
Block 60 of 256
Block 61 of 256
Block 62 of 256
Bl

[(<DomainBlock.DomainBlock at 0x2939548d400>,
  True,
  180,
  -226.47700034140118,
  35792.08427246734),
 (<DomainBlock.DomainBlock at 0x2939548d400>,
  True,
  90,
  -223.16190501859737,
  35268.162242769926),
 (<DomainBlock.DomainBlock at 0x2939548d400>,
  False,
  270,
  -144.84283866098863,
  22891.08414556493),
 (<DomainBlock.DomainBlock at 0x2939548d400>,
  True,
  270,
  -167.2114306506388,
  26426.00166643645),
 (<DomainBlock.DomainBlock at 0x2939548d400>,
  False,
  90,
  -186.72160979641708,
  29509.306416405),
 (<DomainBlock.DomainBlock at 0x2939548d400>,
  True,
  0,
  -189.60638689738198,
  29965.176813384947),
 (<DomainBlock.DomainBlock at 0x2939548d400>,
  False,
  0,
  -187.80572475877312,
  29680.55700501321),
 (<DomainBlock.DomainBlock at 0x2939548d400>,
  True,
  0,
  -187.62061793614,
  29651.35347838953),
 (<DomainBlock.DomainBlock at 0x2939548d400>,
  False,
  90,
  -186.1312867230877,
  29415.93839533583),
 (<DomainBlock.DomainBlock at 0x2939548d400>,
  False,
 